# Census- Employment Status Data

In [1]:
import pandas as pd
import requests

In [2]:
# Census Subject Table API for Employment Status data within Elementary School Districts in California for 2018
url="https://api.census.gov/data/2016/acs/acs1/subject?get=group(S2301)&for=school%20district%20(elementary)&in=state:06"

In [3]:
# Request for HTTP Data from Census API, which is working <Response [200]>
response = requests.get(url)

In [4]:
# Resetting data from API Data for future formatting, lists data in one column
response_json = response.json()

In [5]:
# Places data in a dataframe and drops index column 0 (with headers), which has 52 schools and 702 columns of variables
elementaryschool_df = pd.DataFrame(response_json,columns=response_json[0]).drop(0)

In [1]:
# Print dataframe
#elementaryschool_df

In [7]:
# Print dataframe to csv 
#elementaryschool_df.to_csv("/Users/nataligracia/git/ca-school-enrollment-trend/Elementary2016test.csv")

In [8]:
# View School District to view column titles, 2016 data does not include district name
#elementaryschool_df.loc[elementaryschool_df["NAME"]=="Los Angeles Unified School District, California"]

## _Format Data_

In [9]:
# Census Subject Table Variables for Employment Status data
variableurl = "https://api.census.gov/data/2016/acs/acs1/subject/variables.json"

In [10]:
# Request for HTTP Data from Census API and reset data
variables_json = requests.get(variableurl).json()

In [11]:
# View variable for column title
variables_json["variables"]["S2301_C01_001E"]

{'label': 'Total!!Estimate!!Population 16 years and over',
 'predicateType': 'int',
 'group': 'S2301',
 'limit': 0,
 'attributes': 'S2301_C01_001M,S2301_C01_001EA'}

In [12]:
# Find and replace all columns with variable titles
new_labels = []
for col in elementaryschool_df.columns:
    label = variables_json["variables"].get(col)
    if label is not None:
        label = label['label']
    else:
        if col[-2:] == 'EA':
            label = variables_json["variables"].get(col[:-1])
            label = label['label'] + "||Annotation"
        elif col[-1] == 'M':
            label = variables_json["variables"].get(col[:-1]+'E')
            label = label['label'] + "||MarginOfError"
        elif col[-2:] == 'MA':
            label = variables_json["variables"].get(col[:-2]+'E')
            label = label['label'] + "||MarginOfErrorAnnotation"
    new_labels.append(label)

In [2]:
# Find any columns without titles
#new_labels  

In [14]:
# Change column titles for columns labeled "None" 
assert len(new_labels) == len(elementaryschool_df.columns)

In [15]:
# Confirm the number of columns without titles
sum([1 for x in new_labels if x is None])

2

In [16]:
# Setup new Labels of columns labeled "None"
# new_labels[-283] = 'NAME'
new_labels[-2] = 'STATE'
new_labels[-1] = 'SCHOOL DISTRICT (ELEMENTARY)'

In [17]:
# Create new labels of columns labeled "None"
elementaryschool_df.columns = new_labels

In [3]:
# Find all columns without "Annotation" in column title
#[col for col in elementaryschool_df.columns if "Annotation" not in col]

In [19]:
# Create a new dataframe for data without the columns that have "Annotation" in the title, which is 702 columns
without_annotation = elementaryschool_df[[col for col in elementaryschool_df.columns if "Annotation" not in col]].copy()

In [20]:
without_annotation.head()

,Total!!Estimate!!Population 16 years and over,Total!!Estimate!!Population 16 years and over||MarginOfError,Total!!Estimate!!Population 16 years and over||MarginOfError,Total!!Estimate!!AGE!!16 to 19 years,Total!!Estimate!!AGE!!16 to 19 years||MarginOfError,Total!!Estimate!!AGE!!16 to 19 years||MarginOfError,Total!!Estimate!!AGE!!20 to 24 years,Total!!Estimate!!AGE!!20 to 24 years||MarginOfError,Total!!Estimate!!AGE!!20 to 24 years||MarginOfError,Total!!Estimate!!AGE!!25 to 29 years,...,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Less than high school graduate||MarginOfError,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Less than high school graduate||MarginOfError,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!High school graduate (includes equivalency)||MarginOfError,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!High school graduate (includes equivalency)||MarginOfError,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Some college or associate's degree||MarginOfError,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Some college or associate's degree||MarginOfError,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Bachelor's degree or higher||MarginOfError,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Bachelor's degree or higher||MarginOfError,STATE,SCHOOL DISTRICT (ELEMENTARY)
1,94609,6628,6628,6249,1272,1272,9476,1512,1512,10101,...,3.0,3.0,3.8,3.8,3.4,3.4,2.9,2.9,06,02310
2,161824,6682,6682,12232,1862,1862,16369,2458,2458,21231,...,2.4,2.4,2.8,2.8,2.0,2.0,1.9,1.9,06,02610
3,135056,7316,7316,10371,2312,2312,15582,2591,2591,15996,...,3.8,3.8,7.0,7.0,4.9,4.9,1.9,1.9,06,03630
4,68849,5294,5294,3417,810,810,7086,2470,2470,7581,...,4.4,4.4,6.8,6.8,3.2,3.2,1.9,1.9,06,04800
5,78600,3235,3235,5760,1018,1018,10751,1412,1412,8837,...,2.1,2.1,2.9,2.9,3.8,3.8,1.6,1.6,06,05580


In [21]:
# Find all columns without "MarginOfError" in column title
#[col for col in without_annotation_df.columns if "MarginOfError" not in col]

In [22]:
# Create a new dataframe for data without the columns that have "MarginOfError" in the title, which is 142 columns
withoutmarginerror = without_annotation[[col for col in without_annotation.columns if 'MarginOfError' not in col]].copy()

In [23]:
withoutmarginerror.head()

,Total!!Estimate!!Population 16 years and over,Total!!Estimate!!AGE!!16 to 19 years,Total!!Estimate!!AGE!!20 to 24 years,Total!!Estimate!!AGE!!25 to 29 years,Total!!Estimate!!AGE!!30 to 34 years,Total!!Estimate!!AGE!!35 to 44 years,Total!!Estimate!!AGE!!45 to 54 years,Total!!Estimate!!AGE!!55 to 59 years,Total!!Estimate!!AGE!!60 to 64 years,Total!!Estimate!!AGE!!65 to 74 years,...,Unemployment rate!!Estimate!!POVERTY STATUS IN THE PAST 12 MONTHS!!Below poverty level,Unemployment rate!!Estimate!!POVERTY STATUS IN THE PAST 12 MONTHS!!At or above the poverty level,Unemployment rate!!Estimate!!DISABILITY STATUS!!With any disability,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Less than high school graduate,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!High school graduate (includes equivalency),Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Some college or associate's degree,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Bachelor's degree or higher,STATE,SCHOOL DISTRICT (ELEMENTARY)
1,94609,6249,9476,10101,8587,15806,15543,6829,6535,8738,...,19.0,5.3,12.1,6.1,4.4,8.6,6.8,4.6,06,02310
2,161824,12232,16369,21231,16614,26234,29859,11538,9228,11514,...,9.4,4.8,9.8,4.9,5.6,5.3,4.7,3.8,06,02610
3,135056,10371,15582,15996,13152,20474,18541,11553,9002,11450,...,36.1,6.8,18.8,11.3,11.5,16.6,10.7,2.1,06,03630
4,68849,3417,7086,7581,6535,11708,10889,4060,6091,7047,...,18.9,6.1,28.8,5.9,2.6,10.0,6.5,5.1,06,04800
5,78600,5760,10751,8837,8225,14789,10580,4598,3880,5720,...,10.8,3.0,14.6,4.1,4.3,3.2,6.1,1.0,06,05580


In [24]:
# Find all columns without "Labor Force Participation Rate" in column title
#[col for col in withoutmarginerror.columns if "Labor Force Participation Rate" not in col]

In [25]:
# Create a new dataframe for data without the columns that have "Labor Force Participation Rate" in the title, which is 107 columns
withoutlaborforce = withoutmarginerror[[col for col in withoutmarginerror.columns if 'Labor Force Participation Rate' not in col]].copy()

In [26]:
withoutlaborforce.head()

,Total!!Estimate!!Population 16 years and over,Total!!Estimate!!AGE!!16 to 19 years,Total!!Estimate!!AGE!!20 to 24 years,Total!!Estimate!!AGE!!25 to 29 years,Total!!Estimate!!AGE!!30 to 34 years,Total!!Estimate!!AGE!!35 to 44 years,Total!!Estimate!!AGE!!45 to 54 years,Total!!Estimate!!AGE!!55 to 59 years,Total!!Estimate!!AGE!!60 to 64 years,Total!!Estimate!!AGE!!65 to 74 years,...,Unemployment rate!!Estimate!!POVERTY STATUS IN THE PAST 12 MONTHS!!Below poverty level,Unemployment rate!!Estimate!!POVERTY STATUS IN THE PAST 12 MONTHS!!At or above the poverty level,Unemployment rate!!Estimate!!DISABILITY STATUS!!With any disability,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Less than high school graduate,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!High school graduate (includes equivalency),Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Some college or associate's degree,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Bachelor's degree or higher,STATE,SCHOOL DISTRICT (ELEMENTARY)
1,94609,6249,9476,10101,8587,15806,15543,6829,6535,8738,...,19.0,5.3,12.1,6.1,4.4,8.6,6.8,4.6,06,02310
2,161824,12232,16369,21231,16614,26234,29859,11538,9228,11514,...,9.4,4.8,9.8,4.9,5.6,5.3,4.7,3.8,06,02610
3,135056,10371,15582,15996,13152,20474,18541,11553,9002,11450,...,36.1,6.8,18.8,11.3,11.5,16.6,10.7,2.1,06,03630
4,68849,3417,7086,7581,6535,11708,10889,4060,6091,7047,...,18.9,6.1,28.8,5.9,2.6,10.0,6.5,5.1,06,04800
5,78600,5760,10751,8837,8225,14789,10580,4598,3880,5720,...,10.8,3.0,14.6,4.1,4.3,3.2,6.1,1.0,06,05580


In [27]:
# Find all columns without "Sex" in column title
#[col for col in withoutlaborforce.columns if "SEX" not in col]

In [28]:
# Create a new dataframe for data without the columns that have "Sex" in the title, which is 89 columns
withoutsex = withoutlaborforce[[col for col in withoutlaborforce.columns if 'SEX' not in col]].copy()

In [29]:
withoutsex.head()

,Total!!Estimate!!Population 16 years and over,Total!!Estimate!!AGE!!16 to 19 years,Total!!Estimate!!AGE!!20 to 24 years,Total!!Estimate!!AGE!!25 to 29 years,Total!!Estimate!!AGE!!30 to 34 years,Total!!Estimate!!AGE!!35 to 44 years,Total!!Estimate!!AGE!!45 to 54 years,Total!!Estimate!!AGE!!55 to 59 years,Total!!Estimate!!AGE!!60 to 64 years,Total!!Estimate!!AGE!!65 to 74 years,...,Unemployment rate!!Estimate!!POVERTY STATUS IN THE PAST 12 MONTHS!!Below poverty level,Unemployment rate!!Estimate!!POVERTY STATUS IN THE PAST 12 MONTHS!!At or above the poverty level,Unemployment rate!!Estimate!!DISABILITY STATUS!!With any disability,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Less than high school graduate,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!High school graduate (includes equivalency),Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Some college or associate's degree,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Bachelor's degree or higher,STATE,SCHOOL DISTRICT (ELEMENTARY)
1,94609,6249,9476,10101,8587,15806,15543,6829,6535,8738,...,19.0,5.3,12.1,6.1,4.4,8.6,6.8,4.6,06,02310
2,161824,12232,16369,21231,16614,26234,29859,11538,9228,11514,...,9.4,4.8,9.8,4.9,5.6,5.3,4.7,3.8,06,02610
3,135056,10371,15582,15996,13152,20474,18541,11553,9002,11450,...,36.1,6.8,18.8,11.3,11.5,16.6,10.7,2.1,06,03630
4,68849,3417,7086,7581,6535,11708,10889,4060,6091,7047,...,18.9,6.1,28.8,5.9,2.6,10.0,6.5,5.1,06,04800
5,78600,5760,10751,8837,8225,14789,10580,4598,3880,5720,...,10.8,3.0,14.6,4.1,4.3,3.2,6.1,1.0,06,05580


In [30]:
# Find all columns without "Poverty Status" in column title
#[col for col in withoutsex.columns if "POVERTY STATUS" not in col]

In [31]:
# Create a new dataframe for data without the columns that have "Poverty Status" in the title, which is 83 columns
withoutps = withoutsex[[col for col in withoutsex.columns if 'POVERTY STATUS' not in col]].copy()

In [32]:
withoutps.head()

,Total!!Estimate!!Population 16 years and over,Total!!Estimate!!AGE!!16 to 19 years,Total!!Estimate!!AGE!!20 to 24 years,Total!!Estimate!!AGE!!25 to 29 years,Total!!Estimate!!AGE!!30 to 34 years,Total!!Estimate!!AGE!!35 to 44 years,Total!!Estimate!!AGE!!45 to 54 years,Total!!Estimate!!AGE!!55 to 59 years,Total!!Estimate!!AGE!!60 to 64 years,Total!!Estimate!!AGE!!65 to 74 years,...,"Unemployment rate!!Estimate!!White alone, not Hispanic or Latino",Unemployment rate!!Estimate!!Population 20 to 64 years,Unemployment rate!!Estimate!!DISABILITY STATUS!!With any disability,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Less than high school graduate,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!High school graduate (includes equivalency),Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Some college or associate's degree,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Bachelor's degree or higher,STATE,SCHOOL DISTRICT (ELEMENTARY)
1,94609,6249,9476,10101,8587,15806,15543,6829,6535,8738,...,6.3,6.6,12.1,6.1,4.4,8.6,6.8,4.6,06,02310
2,161824,12232,16369,21231,16614,26234,29859,11538,9228,11514,...,6.2,5.2,9.8,4.9,5.6,5.3,4.7,3.8,06,02610
3,135056,10371,15582,15996,13152,20474,18541,11553,9002,11450,...,9.0,12.6,18.8,11.3,11.5,16.6,10.7,2.1,06,03630
4,68849,3417,7086,7581,6535,11708,10889,4060,6091,7047,...,8.8,6.6,28.8,5.9,2.6,10.0,6.5,5.1,06,04800
5,78600,5760,10751,8837,8225,14789,10580,4598,3880,5720,...,3.1,3.6,14.6,4.1,4.3,3.2,6.1,1.0,06,05580


In [33]:
# Find all columns without "Disability Status" in column title
#[col for col in withoutps.columns if "DISABILITY STATUS" not in col]

In [34]:
# Create a new dataframe for data without the columns that have "Disability Status" in the title, which is 80 columns
withoutds = withoutps[[col for col in withoutps.columns if 'DISABILITY STATUS' not in col]].copy()

In [35]:
withoutds.head()

,Total!!Estimate!!Population 16 years and over,Total!!Estimate!!AGE!!16 to 19 years,Total!!Estimate!!AGE!!20 to 24 years,Total!!Estimate!!AGE!!25 to 29 years,Total!!Estimate!!AGE!!30 to 34 years,Total!!Estimate!!AGE!!35 to 44 years,Total!!Estimate!!AGE!!45 to 54 years,Total!!Estimate!!AGE!!55 to 59 years,Total!!Estimate!!AGE!!60 to 64 years,Total!!Estimate!!AGE!!65 to 74 years,...,Unemployment rate!!Estimate!!Hispanic or Latino origin (of any race),"Unemployment rate!!Estimate!!White alone, not Hispanic or Latino",Unemployment rate!!Estimate!!Population 20 to 64 years,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Less than high school graduate,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!High school graduate (includes equivalency),Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Some college or associate's degree,Unemployment rate!!Estimate!!EDUCATIONAL ATTAINMENT!!Population 25 to 64 years!!Bachelor's degree or higher,STATE,SCHOOL DISTRICT (ELEMENTARY)
1,94609,6249,9476,10101,8587,15806,15543,6829,6535,8738,...,7.4,6.3,6.6,6.1,4.4,8.6,6.8,4.6,06,02310
2,161824,12232,16369,21231,16614,26234,29859,11538,9228,11514,...,5.5,6.2,5.2,4.9,5.6,5.3,4.7,3.8,06,02610
3,135056,10371,15582,15996,13152,20474,18541,11553,9002,11450,...,14.5,9.0,12.6,11.3,11.5,16.6,10.7,2.1,06,03630
4,68849,3417,7086,7581,6535,11708,10889,4060,6091,7047,...,-999999999.0,8.8,6.6,5.9,2.6,10.0,6.5,5.1,06,04800
5,78600,5760,10751,8837,8225,14789,10580,4598,3880,5720,...,4.1,3.1,3.6,4.1,4.3,3.2,6.1,1.0,06,05580


In [36]:
# Find all columns without "Educational Attainment" in column title
#[col for col in withoutds.columns if "EDUCATIONAL ATTAINMENT" not in col]

In [37]:
# Create a new dataframe for data without the columns that have "Educational Attainment" in the title, which is 65 columns
withoutea = withoutds[[col for col in withoutds.columns if 'EDUCATIONAL ATTAINMENT' not in col]].copy()

In [38]:
withoutea.head()

,Total!!Estimate!!Population 16 years and over,Total!!Estimate!!AGE!!16 to 19 years,Total!!Estimate!!AGE!!20 to 24 years,Total!!Estimate!!AGE!!25 to 29 years,Total!!Estimate!!AGE!!30 to 34 years,Total!!Estimate!!AGE!!35 to 44 years,Total!!Estimate!!AGE!!45 to 54 years,Total!!Estimate!!AGE!!55 to 59 years,Total!!Estimate!!AGE!!60 to 64 years,Total!!Estimate!!AGE!!65 to 74 years,...,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!American Indian and Alaska Native alone,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Asian alone,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Native Hawaiian and Other Pacific Islander alone,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Some other race alone,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Two or more races,Unemployment rate!!Estimate!!Hispanic or Latino origin (of any race),"Unemployment rate!!Estimate!!White alone, not Hispanic or Latino",Unemployment rate!!Estimate!!Population 20 to 64 years,STATE,SCHOOL DISTRICT (ELEMENTARY)
1,94609,6249,9476,10101,8587,15806,15543,6829,6535,8738,...,-999999999.0,4.0,-999999999.0,5.3,-999999999.0,7.4,6.3,6.6,06,02310
2,161824,12232,16369,21231,16614,26234,29859,11538,9228,11514,...,-999999999.0,5.5,-999999999.0,-999999999.0,-999999999.0,5.5,6.2,5.2,06,02610
3,135056,10371,15582,15996,13152,20474,18541,11553,9002,11450,...,-999999999.0,-999999999.0,-999999999.0,15.3,-999999999.0,14.5,9.0,12.6,06,03630
4,68849,3417,7086,7581,6535,11708,10889,4060,6091,7047,...,-999999999.0,5.9,-999999999.0,-999999999.0,-999999999.0,-999999999.0,8.8,6.6,06,04800
5,78600,5760,10751,8837,8225,14789,10580,4598,3880,5720,...,-999999999.0,-999999999.0,-999999999.0,-999999999.0,-999999999.0,4.1,3.1,3.6,06,05580


In [39]:
# Find all columns without "Age" in column title
#[col for col in withoutea.columns if "AGE" not in col]

In [40]:
# Create a new dataframe for data without the columns that have "Age" in the title, which is 35 columns
withoutage = withoutea[[col for col in withoutea.columns if 'AGE' not in col]].copy()

In [4]:
#withoutage

In [42]:
# Format to rename School District Name
#def cleandistrict(NAME):
    #return NAME.replace(", California","")

In [43]:
# Apply formatting condition to School District Name
#withoutage['NAME'].apply(cleandistrict)

In [44]:
# Create new School District name column with formatting titled District
#withoutage['District']= withoutage['NAME'].apply(cleandistrict)

In [45]:
# Find length of Geographt column contents, which is 16
#withoutage['Geography'].apply(len).unique()

# Pull a geography
#geo = "9700000US0622710"

# Find NCESDist ID
#geo.split("US")[1]

In [46]:
# Format to seperate Census Geography code (the state and district ID combined), 
#def splitGeo(geo):
    #return geo.split("US")[1]

In [47]:
# Apply formatting condition to Geography
#withoutage['Geography'].apply(splitGeo)

In [48]:
# Create new Geography name column with formatting titled NCESDist
#withoutage['NCESDist']= withoutage['Geography'].apply(splitGeo)

In [49]:
# Create new column with combined state and district ID
withoutage['NCESDist'] = withoutage['STATE'].astype(str) + withoutage['SCHOOL DISTRICT (ELEMENTARY)']

In [50]:
# Add Year column
withoutage['year']= "2016"

In [51]:
withoutage.head()

,Total!!Estimate!!Population 16 years and over,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!White alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Black or African American alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!American Indian and Alaska Native alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Asian alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Native Hawaiian and Other Pacific Islander alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Some other race alone,Total!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Two or more races,Total!!Estimate!!Hispanic or Latino origin (of any race),"Total!!Estimate!!White alone, not Hispanic or Latino",...,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Native Hawaiian and Other Pacific Islander alone,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Some other race alone,Unemployment rate!!Estimate!!RACE AND HISPANIC OR LATINO ORIGIN!!Two or more races,Unemployment rate!!Estimate!!Hispanic or Latino origin (of any race),"Unemployment rate!!Estimate!!White alone, not Hispanic or Latino",Unemployment rate!!Estimate!!Population 20 to 64 years,STATE,SCHOOL DISTRICT (ELEMENTARY),NCESDist,year
1,94609,26440,-999999999,-999999999,29185,-999999999,30955,-999999999,50904,9293,...,-999999999.0,5.3,-999999999.0,7.4,6.3,6.6,06,02310,0602310,2016
2,161824,116572,-999999999,-999999999,25429,-999999999,-999999999,-999999999,97414,32838,...,-999999999.0,-999999999.0,-999999999.0,5.5,6.2,5.2,06,02610,0602610,2016
3,135056,85941,-999999999,-999999999,-999999999,-999999999,27228,-999999999,81189,36077,...,-999999999.0,15.3,-999999999.0,14.5,9.0,12.6,06,03630,0603630,2016
4,68849,13905,-999999999,-999999999,46246,-999999999,-999999999,-999999999,-999999999,10470,...,-999999999.0,-999999999.0,-999999999.0,-999999999.0,8.8,6.6,06,04800,0604800,2016
5,78600,61841,-999999999,-999999999,-999999999,-999999999,-999999999,-999999999,55589,15045,...,-999999999.0,-999999999.0,-999999999.0,4.1,3.1,3.6,06,05580,0605580,2016


In [52]:
# Print dataframe to csv
#withoutage.to_csv("/Users/nataligracia/git/ca-school-enrollment-trend/Elementary2016.csv")

In [53]:
# Rearrange columns in list
#['NCESDist','District'] + list(without_annotation_df.columns[2:])

In [54]:
# Rearrange columns in dataframe
#final = without_annotation_df[['NCESDist','District','Geography','NAME'] + list(without_annotation_df.columns[2:])]
#final